## Threads
There are two primary ways to create a thread. 1) Extend the `Thread` class and override the run method.

In [ ]:
// Worker cannot inherit some functionality from other classes
public class Worker extends Thread {
    @Override
    public void run(){
        System.out.println("Worker thread running");
    }
}

Worker worker = new Worker();

2) The preferred way is to pass a `Runnable` instance to `Thread` constructor:

In [ ]:
Runnable runnable = new Runnable{
    public void run(){
        System.out.println("Worker thread running");
    }
}

Thread worker = new Thread(runnable);

**Starting a thread:** use the `start` method of the `Thread` class. The `isAlive` method can tell if the thread is currently running or not. `isAlive` returns false if the thread has not been started or the thread has been terminated.  

**Stopping a thread:** is not as straight forward as starting it. The only way to terminate a thread is to arrange for its `run` method to complete. The thread class has a `stop` method, but it has been deprecated. It has been deprecated because stopping a thread releases all the locks it may have. If the thread was inside a critical section when stopped, it would lead to corrupted state. Other threads which access that critical section might be working with corrupted state.

Thread throwing an unchecked exception can also terminate the thread. Two approaches for stopping a thread: 
1) setting a flag

In [ ]:
public class Worker extends Thread {
    // done must be set as volatile for this to work
    private volatile boolean done = false;
    
    @Override
    public void run(){
        while(!done){
            // Do something
        }
    }
    
    // Other methods
}

An external code can stop this thread by setting done as true. But the thread will only stop when the execution reaches the top of the loop (and not immediately).  
2) Using interrupts: we can call `interrupt` method of any thread. It has two effects. First, it sets internal interrupted flag of the thread to true. Second, it causes any blocking method like `sleep`, `join`, `wait` to stop blocking and throw `InterruptedException`.

In [ ]:
public class Worker extends Thread{
    
    @Override
    public void run(){
        while(!isInterrupted()){
            // Do something
        }
    }
    
    // Other methods
}

Worker worker = new Worker();
worker.start();
worker.interrupt();

**Pausing a thread:** the Thread class has `suspend` and `resume` methods which have been deprecated (due to possibility of deadlock). It is still possible for a thread to suspend its own execution for a specified amount of time using `sleep` method. The least amount of time a thread can sleep on most Java implementations is 20 to 50 ms.

In [6]:
public class DelayedRandomGenerator extends Thread {
    public void run() {
        Random random = new Random();
        // If DelayedRandomGenerator implemented Runnable instead, use Thread.currentThread().isInterrupted()
        while (!isInterrupted()) {
            System.out.print(random.nextInt() + "\t");
            try {
                sleep(200);
            } catch (InterruptedException e) {
                break;
            }
        }
    }
}

DelayedRandomGenerator generator = new DelayedRandomGenerator();
generator.start();

try {
    Thread.sleep(1000);
} catch (InterruptedException e) {
    System.exit(0);
}        

generator.interrupt();

1152488192	2140810343	1285862342	1650368227	-1184878083	1039227028	278517272	-1543200418	401414008	-531716027	

## Thread Notification
`wait` and `notify` semantics allows threads to communicate among each other. The idea is that one thread needs a certain condition to exist and assumes that another thread will create that condition. When another thread creates that condition it notifies the first thread that has been waiting for the condition.

The semantics is achieved using the following methods that are present in the `Object` class:
- `wait`: waits for the condition to occur. The current thread lets go of the lock
- `wait(long timeout)`: similar to the first except that it comes back into contention for the lock acquisition after the timeout
- `notify`: signals another thread that the condition has happened and it should stop waiting. The notified thread would then try to acquire lock.
- `notifyAll`: signals all the waiting threads (waiting on the same lock). One of the waiting thread would ultimately acquire lock.

All the above methods must be called from within synchronization method or block. Why? Because of  inherent race condition:
1. Thread A tests a condition and discovers it must wait.
2. Thread B sets the condition and calls notify() to inform A to resume execution. Because A is not yet waiting, nothing happens.
3. Thread A waits, by calling wait().
4. Because of the prior notify() call, A waits indefinitely.

It is mandatory for `wait` and `notify` to hold the locks for the object they are operating on. The `wait` method releases the lock prior to waiting and reacquires the lock prior to returning from the `wait` method.

If a thread receives a notification, it is not guaranteed that the condition is set correctly. Prior to calling the `wait` method, thread should test for the condition while holding the lock. Upon returning from the `wait` method the thread should retest the condition (to check whether it should wait again). Why? Suppose three threads A, B and C.
1. Thread A calls a method that acquires the synchronization lock.
2. Thread A examines a state flag and determines that the data is not in the desired state.
3. Thread A calls the `wait()` method, which frees the lock.
4. Thread B calls a method that acquires the same synchronization lock.
5. Thread C calls a method that blocks waiting for the lock.
6. Thread B sets the state flag and calls the `notifyAll()` method.
7. Thread B finishes its method and frees the lock.
8. Thread C acquires the lock and proceeds to process the data; it sees that the data is in the desired state, so it processes the data and resets the state flag.
9. Thread C exits without needing to wait.
10. Thread A receives the notification and wakes up.

Threads that are waiting can be considered as consumers - there is no guarantee that when the thread wakes up the data has not been processed by another consumer. When the thread wakes up, it cannot assume that the condition it has been waiting for would still be true. Waiting thread must provide an option to check the state and return back to waiting state in case the notification has already been handled. This is something that calls to `wait` inside a for loop can achieve.

In [ ]:
// Consider that this object is accessed by two consumer threads and one producer
class SharedData {
    private String data;

    public synchronized void consume() throws InterruptedException {
        // Check for condition before waiting
        if (data == null) {
            wait();
        }

        // Process the data
        System.out.println(data);
        // Signal that data has been processed
        data = null;
    }

    public synchronized void produce() {
        data = "Hello world!";
        notifyAll();
    }
}

The output in this case is:
```
Hello world!
null
```

The thread that calls `wait` (the waiting thread) and the thread that calls notify (the notification thread) must compete for the same lock. Either thread must call `wait` or `notify` via the same object on which they enter their synchronized contexts because wait tightly integrates with the lock. Below example elaborates this using another object as lock:

In [ ]:
class Data {
    private String data;
    private Object lock = new Object();
    private boolean sent = true;

    public void send(String data) {
        synchronized (lock) {
            while (!sent) {
                try {
                    // Notice that wait is called on the lock object
                    lock.wait();
                } catch (InterruptedException e) {
                    e.printStackTrace();
                }
            }

            sent = false;
            this.data = data;

            // Notify is also called on the same lock
            lock.notifyAll();
        }
    }

    public String receive(){
        synchronized (lock) {
            while (sent) {
                try {
                    lock.wait();
                } catch (InterruptedException e) {
                    e.printStackTrace();
                }
            }

            sent = true;
            lock.notifyAll();

            return this.data;
        }
    }
}

Wait/notify forms the basis of producer consumer pattern.

In [ ]:
class Common {
    private int number;
    private boolean writeable = true;

    public synchronized void setNumber(int number) {
        // If writeable is false, this means the consumer
        // has not consumed the data, wait
        // Also notice that this is inside a while loop and not a simple if condition
        while (!writeable) {
            try {
                wait();
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }

        writeable = false;
        this.number = number;
        System.out.println("Number " + this.number + " set");

        notify(); // If we miss this all consumers will continue
                  // waiting. Here we notify all waiting consumers that
                  // that new data is ready
    }

    public synchronized int getNumber() {
        // Producer has not produced new data, so better wait
        while (writeable) {
            try {
                wait();
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }

        writeable = true;
        notify(); // Notify producer that data has been consumed
                  // they can stop waiting (after this method
                  // has been executed)

        System.out.println("Number " + this.number + " read");
        return this.number;
    }
}

class Producer extends Thread {
    private Common common;

    public Producer(Common common) {
        this.common = common;
    }

    @Override
    public void run() {
        // Produce even numbers sequentially
        for (int i = 0; i < 100; i++) {
            if (i % 2 == 0) {
                common.setNumber(i);
            }
        }
    }
}

class Consumer extends Thread {
    private Common common;

    public Consumer(Common common) {
        this.common = common;
    }

    @Override
    public void run() {
        // Consume all generated numbers
        for (int i = 0; i < 50; i++) {
            common.getNumber();
        }
    }
}